# Heritability Analysis

This note demonstrates the workflow for a typical heritability analysis in genetics, using a sample data set `cg10k` with **6,670** individuals and **630,860** SNPs. Person IDs and phenotype names are masked for privacy. Here `cg10k.bed`, `cg10k.bim`, and `cg10k.fam` is a set of Plink files in binary format. `cg10k_traits.txt` contains the phenotype data of 6,670 individuals.

In [1]:
;ls cg10k*.*

cg10k.bed
cg10k.bim
cg10k.fam
cg10k_traits.txt


## Read in binary SNP data

We will use the [`SnpArrays`](https://github.com/Hua-Zhou/SnpArrays.jl) package to read in binary SNP data and compute the empirical kinship matrix. Issue `Pkg.clone("git@github.com:Hua-Zhou/SnpArrays.jl.git")` within `Julia` to install the `SnpArrays` package.

In [2]:
#Pkg.clone("git@github.com:Hua-Zhou/SnpArrays.jl.git")
using SnpArrays
# read in genotype data from Plink binary file (~50 secs on my laptop)
@time copd = SnpArray("cg10k")

 50.915927 seconds (503.93 k allocations: 1.003 GB, 0.04% gc time)


6670x630860 SnpArrays.SnpArray{2}:
 (false,true)   (false,true)   (true,true)   …  (true,true)    (true,true) 
 (true,true)    (true,true)    (false,true)     (false,true)   (true,false)
 (true,true)    (true,true)    (false,true)     (true,true)    (true,true) 
 (true,true)    (true,true)    (true,true)      (false,true)   (true,true) 
 (true,true)    (true,true)    (true,true)      (true,true)    (false,true)
 (false,true)   (false,true)   (true,true)   …  (true,true)    (true,true) 
 (false,false)  (false,false)  (true,true)      (true,true)    (true,true) 
 (true,true)    (true,true)    (true,true)      (true,true)    (false,true)
 (true,true)    (true,true)    (false,true)     (true,true)    (true,true) 
 (true,true)    (true,true)    (true,true)      (false,true)   (true,true) 
 (true,true)    (true,true)    (false,true)  …  (true,true)    (true,true) 
 (false,true)   (false,true)   (true,true)      (true,true)    (false,true)
 (true,true)    (true,true)    (true,true)      (true

## Summary statistics of SNP data

In [3]:
people, snps = size(copd)

(6670,630860)

In [4]:
# summary statistics (~50 secs on my laptop)
@time maf, minor_allele, missings_by_snp, missings_by_person = summarize(copd)

 42.989971 seconds (42.38 k allocations: 11.660 MB)


([0.169916,0.17099,0.114026,0.268694,0.219265,0.23935,0.190612,0.20201,0.0271609,0.299714  …  0.207996,0.448012,0.295465,0.142654,0.170915,0.281428,0.0611354,0.0524738,0.13931,0.132413],Bool[true,true,true,true,true,true,true,true,true,true  …  true,true,true,true,true,true,true,true,true,true],[2,0,132,77,0,27,2,2,6,27  …  4,5,11,0,0,4,29,0,5,43],[364,1087,1846,1724,1540,248,215,273,273,224  …  1353,692,901,1039,685,304,223,1578,2059,1029])

In [5]:
# 5 number summary and average MAF (minor allele frequencies)
quantile(maf, [0.0 .25 .5 .75 1.0]), mean(maf)

(
1x5 Array{Float64,2}:
 0.00841726  0.124063  0.236953  0.364253  0.5,

0.24536516625042462)

In [58]:
using Plots
#pyplot()
gr()

histogram(maf, xlab = "Minor Allele Frequency (MAF)", label = "MAF")

[Plots.jl] Initializing backend: gr


INFO: Recompiling stale cache file /Users/hzhou3/.julia/lib/v0.4/GR.ji for module GR.


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 
 
 
 Minor Allele Frequency (MAF) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 MAF

In [8]:
# total number of missing genotypes
sum(missings_by_snp)

5524131

In [59]:
# proportion of SNPs with maf > 0.05
countnz(maf .≥ 0.05) / length(maf)

0.9277193038075009

## Empirical kinship matrix

We estimate empirical kinship based on all SNPs by genetic relation matrix (GRM) and method of moment (MoM).

In [10]:
# GRM using all SNPs (~10 mins on my laptop)
srand(123)
@time Φgrm = grm(copd)

623.389668 seconds (4.21 G allocations: 64.486 GB, 1.14% gc time)


6670x6670 Array{Float64,2}:
  0.502916      0.00329978   -0.000116213  …  -6.46286e-5   -0.00281229 
  0.00329978    0.49892      -0.00201992       0.000909871   0.00345573 
 -0.000116213  -0.00201992    0.493632         0.000294565  -0.000349854
  0.000933977  -0.00320391   -0.0018611       -0.00241682   -0.00127078 
 -7.75429e-5   -0.0036075     0.00181442       0.00213976   -0.00158382 
  0.00200371    0.000577386   0.0025455    …   0.000943753  -1.82994e-6 
  0.000558503   0.00241421   -0.0018782        0.001217     -0.00123924 
 -0.000659495   0.00319987   -0.00101496       0.00353646   -0.00024093 
 -0.00102619   -0.00120448   -0.00055462       0.00175586    0.00181899 
 -0.00136838    0.00211996    0.000119128     -0.00147305   -0.00105239 
 -0.00206144    0.000148818  -0.000475177  …  -0.000265522  -0.00106123 
  0.000951016   0.00167042    0.00183545      -0.000703658  -0.00313334 
  0.000330442  -0.000904147   0.00301478       0.000754772  -0.00127413 
  ⋮                    

In [15]:
# MoM using all SNPs
srand(123)
@time Φmom = grm(copd; method = :MoM)

603.181891 seconds (4.21 G allocations: 64.478 GB, 1.02% gc time)


6670x6670 Array{Float64,2}:
  0.496512      0.000821702  -0.000437973  …   0.000582675  -0.00523526 
  0.000821702   0.499079     -0.00103315       0.00268612    0.00332432 
 -0.000437973  -0.00103315    0.500996         0.00358008    0.00200011 
  0.000310183  -0.00306489    0.00111332       0.000341257  -5.07485e-5 
  0.00150532   -0.000622024   0.00771765       0.00785389    0.00224631 
  0.00334344    0.00343905    0.00687627   …   0.00483258    0.0032072  
 -0.000397339   0.00201923    0.000723701      0.00338408   -0.000777392
 -0.00311508    0.0022965     0.00103205       0.00647709    0.000568333
 -0.00413573   -0.00235736    0.00138103       0.00345818    0.00161527 
 -0.00151838    0.0035251     0.00477761       0.00284388    0.000697408
 -0.00279718    0.000577894   0.00289407   …   0.00318329    9.2668e-5  
  0.00125673    0.00191167    0.00499034       0.00233236   -0.00157096 
 -0.000619634   0.000494234   0.00731608       0.00349881   -0.000124847
  ⋮                    

## Save results

We don't want to re-compute SnpArray and empirical kinship matrices again and again for heritibility analysis.

In [8]:
#using JLD
#@save "cg10k.jld"
#whos()

                 #163#wsession    280 bytes  JLD.JldWriteSession
                    ArrayViews    183 KB     Module
                          Base  33051 KB     Module
                         Blosc     37 KB     Module
                      Calculus    166 KB     Module
                    ColorTypes    314 KB     Module
                        Colors    748 KB     Module
                        Compat    306 KB     Module
                          Core   6620 KB     Module
                    DataArrays    915 KB     Module
                    DataFrames   1944 KB     Module
                 Distributions   1319 KB     Module
                        Docile    414 KB     Module
                   DualNumbers     58 KB     Module
                        FileIO    533 KB     Module
             FixedPointNumbers     33 KB     Module
               FixedSizeArrays    157 KB     Module
                            GR    414 KB     Module
                          GZip    637 KB     Module

To load workspace

In [1]:
using SnpArrays, JLD, DataFrames, VarianceComponentModels, Plots
gr()
@load "cg10k.jld"
whos()

                    ArrayViews    183 KB     Module
                          Base  29117 KB     Module
                         Blosc     37 KB     Module
                      Calculus    166 KB     Module
                    ColorTypes    248 KB     Module
                        Colors    576 KB     Module
                        Compat    247 KB     Module
                          Core   3399 KB     Module
                    DataArrays    865 KB     Module
                    DataFrames   1506 KB     Module
                 Distributions   1317 KB     Module
                        Docile    414 KB     Module
                   DualNumbers     57 KB     Module


                        FileIO    533 KB     Module
             FixedPointNumbers     32 KB     Module
               FixedSizeArrays    157 KB     Module
                          GZip    578 KB     Module
                          Grid    182 KB     Module
                          HDF5   2521 KB     Module
                        IJulia    402 KB     Module
                IPythonDisplay     33 KB     Module
                         Ipopt     26 KB     Module
              IterativeSolvers    485 KB     Module
                           JLD    699 KB     Module
                          JSON    194 KB     Module
                        KNITRO    273 KB     Module
                 KernelDensity     95 KB     Module
                          Main 2474124 KB     Module
                  MathProgBase    304 KB     Module
                      Measures     14 KB     Module
                       NaNMath    106 KB     Module
                        Nettle     57 KB     Module
           

## Phenotypes

Read in the phenotype data and compute descriptive statistics.

In [3]:
using DataFrames

cg10k_trait = readtable("cg10k_traits.txt"; 
    separator = ' ',
    names = [:FID; :IID; :Trait1; :Trait2; :Trait3; :Trait4; :Trait5; :Trait6; 
             :Trait7; :Trait8; :Trait9; :Trait10; :Trait11; :Trait12; :Trait13],  
    eltypes = [UTF8String; UTF8String; Float64; Float64; Float64; Float64; Float64; 
               Float64; Float64; Float64; Float64; Float64; Float64; Float64; Float64])

,FID,IID,Trait1,Trait2,Trait3,Trait4,Trait5,Trait6,Trait7,Trait8,Trait9,Trait10,Trait11,Trait12,Trait13
1,10002K,10002K,-1.81573145026234,-0.94615046147283,1.11363077580442,-2.09867121119159,0.744416614111748,0.00139171884080131,0.934732480409667,-1.22677315418103,1.1160784277875,-0.4436280335029,0.824465656443384,-1.02852542216546,-0.394049201727681
2,10004O,10004O,-1.24440094378729,0.109659992547179,0.467119394241789,-1.62131304097589,1.0566758355683,0.978946979419181,1.00014633946047,0.32487427140228,1.16232175219696,2.6922706948705,3.08263672461047,1.09064954786013,0.0256616415357438
3,10005Q,10005Q,1.45566914502305,1.53866932923243,1.09402959376555,0.586655272226893,-0.32796454430367,-0.30337709778827,-0.0334354881314741,-0.464463064285437,-0.3319396273436,-0.486839089635991,-1.10648681564373,-1.42015780427231,-0.687463456644413
4,10006S,10006S,-0.768809276698548,0.513490885514249,0.244263028382142,-1.31740254475691,1.19393774326845,1.17344127734288,1.08737426675232,0.536022583732261,0.802759240762068,0.234159411749815,0.394174866891074,-0.767365892476029,0.0635385761884935
5,10009Y,10009Y,-0.264415132547719,-0.348240421825694,-0.0239065083413606,0.00473915802244948,1.25619191712193,1.2038883667631,1.29800739042627,0.310113660247311,0.626159861059352,0.899289129831224,0.54996783350812,0.540687809542048,0.179675416046033
6,10010J,10010J,-1.37617270917293,-1.47191967744564,0.291179894254146,-0.803110740704731,-0.264239977442213,-0.260573027836772,-0.165372266287781,-0.219257294118362,1.04702422290318,-0.0985815534616482,0.947393438068448,0.594014812031438,0.245407436348479
7,10011L,10011L,0.1009416296374,-0.191615722103455,-0.567421321596677,0.378571487240382,-0.246656179817904,-0.608810750053858,0.189081058215596,-1.27077787326519,-0.452476199143965,0.702562877297724,0.332636218957179,0.0026916503626181,0.317117176705358
8,10013P,10013P,-0.319818276367464,1.35774480657283,0.818689545938528,-1.15565531644352,0.63448368102259,0.291461908634679,0.933323714954726,-0.741083289682492,0.647477683507572,-0.970877627077966,0.220861165411304,0.852512250237764,-0.225904624283945
9,10014R,10014R,-0.288334173342032,0.566082538090752,0.254958336116175,-0.652578302869714,0.668921559277347,0.978309199170558,0.122862966041938,1.4790926378214,0.0672132424173449,0.0795903917527827,0.167532455243232,0.246915579442139,0.539932616458363
10,10015T,10015T,-1.15759732583991,-0.781198583545165,-0.595807759833517,-1.00554980260402,0.789828885933321,0.571058413379044,0.951304176233755,-0.295962982984816,0.99042002479707,0.561309366988983,0.733100030623233,-1.73467772245684,-1.35278484330654


In [69]:
#describe(cg10k_trait)

In [62]:
Y = convert(Matrix{Float64}, cg10k_trait[:, 3:15])
histogram(Y, layout = 13)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 
 - 
 
 
 2 
 
 
 0 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 
 - 
 
 
 2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y2 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 500 
 
 
 1000 
 
 
 
 0 
 
 
 5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 
 - 
 
 
 2 
 
 
 0 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 
 - 
 
 
 4 
 
 
 - 
 
 
 2 
 
 
 0 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 
 - 
 
 
 2 
 
 
 0 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 
 - 
 
 
 4 
 
 
 - 
 
 
 2 
 
 
 0 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 500 
 
 
 1000 
 
 
 
 - 
 
 
 5 
 
 
 0 
 
 
 5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y8 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 
 - 
 
 
 4 
 
 
 - 
 
 
 2 
 
 
 0 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y9 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 500 
 
 
 1000 
 
 
 
 0 
 
 
 5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y10 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 
 - 
 
 
 2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y11 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1000 
 
 
 2000 
 
 
 
 - 
 
 
 5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y12 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1000 
 
 
 2000 
 
 
 3000 
 
 
 
 0 
 
 
 10 
 
 
 20

## Pre-processing data for heritability analysis

To prepare variance component model fitting, we form an instance of `VarianceComponentVariate`. The two variance components are $(2\Phi, I)$.

In [63]:
using VarianceComponentModels

# form data as VarianceComponentVariate
cg10kdata = VarianceComponentVariate(Y, (2Φgrm, eye(size(Y, 1))))
fieldnames(cg10kdata)

3-element Array{Symbol,1}:
 :Y
 :X
 :V

In [64]:
cg10kdata

VarianceComponentModels.VarianceComponentVariate{Float64,2,Array{Float64,2},Array{Float64,2},Array{Float64,2}}(6670x13 Array{Float64,2}:
 -1.81573   -0.94615     1.11363    …   0.824466  -1.02853     -0.394049 
 -1.2444     0.10966     0.467119       3.08264    1.09065      0.0256616
  1.45567    1.53867     1.09403       -1.10649   -1.42016     -0.687463 
 -0.768809   0.513491    0.244263       0.394175  -0.767366     0.0635386
 -0.264415  -0.34824    -0.0239065      0.549968   0.540688     0.179675 
 -1.37617   -1.47192     0.29118    …   0.947393   0.594015     0.245407 
  0.100942  -0.191616   -0.567421       0.332636   0.00269165   0.317117 
 -0.319818   1.35774     0.81869        0.220861   0.852512    -0.225905 
 -0.288334   0.566083    0.254958       0.167532   0.246916     0.539933 
 -1.1576    -0.781199   -0.595808       0.7331    -1.73468     -1.35278  
  0.740569   1.40874     0.73469    …  -0.712244  -0.313346    -0.345419 
 -0.675892   0.279893    0.267916      -0.809014 

Before fitting the variance component model, we pre-compute the eigen-decomposition of $2\Phi$, the rotated responses, and the constant part in log-likelihood, and store them as a `TwoVarCompVariateRotate` instance, which is re-used in various variane component estimation procedures.

In [7]:
@time cg10kdata_rotated = TwoVarCompVariateRotate(cg10kdata)
fieldnames(cg10kdata_rotated)

 54.869903 seconds (2.14 M allocations: 1.091 GB, 0.17% gc time)


4-element Array{Symbol,1}:
 :Yrot    
 :Xrot    
 :eigval  
 :logdetV2

## Heritability of single traits

Let's invoke Fisher scoring algorithm to fit variance component model for each single trait.

In [65]:
# heritability from single trait analysis
hST = zeros(13)
# standard errors of estimated heritability
hST_se = zeros(13)
# additive genetic effects
σ2a = zeros(13)
# enviromental effects
σ2e = zeros(13)

@time for trait in 1:13
    println(names(cg10k_trait)[trait + 2])
    # form data set for trait j
    traitj_data = TwoVarCompVariateRotate(cg10kdata_rotated.Yrot[:, trait], cg10kdata_rotated.Xrot, 
        cg10kdata_rotated.eigval, cg10kdata_rotated.logdetV2)
    # initialize model parameters
    traitj_model = VarianceComponentModel(traitj_data)
    # estimate variance components
    _, _, _, Σcov, _, _ = mle_fs!(traitj_model, traitj_data; solver=:Knitro, verbose=false)
    σ2a[trait] = traitj_model.Σ[1][1]
    σ2e[trait] = traitj_model.Σ[2][1]
    @show σ2a[trait], σ2e[trait]
    # estimate heritability from variance component estimates
    h, hse = heritability(traitj_model.Σ, Σcov)
    hST[trait] = h[1, 1]
    hST_se[trait] = hse[1, 1]
end

Trait1
(σ2a[trait],σ2e[trait]) = (0.26104124348442725,0.7356884322639051)
Trait2
(σ2a[trait],σ2e[trait]) = 

### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.


(0.1887416044532389,0.8106898711546213)
Trait3
(σ2a[trait],σ2e[trait]) = (0.3185719312493481,0.6801458828228929)
Trait4
(σ2a[trait],σ2e[trait]) = 

### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.
### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.


(0.2655690169564886,0.7303588329793687)
Trait5
(σ2a[trait],σ2e[trait]) = (0.2812332027554395,0.716798913627366)
Trait6
(σ2a[trait],σ2e[trait]) = 

...

### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.
### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.


(0.08298664768480632,0.9158035237004613)
Trait13


### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.


(σ2a[trait],σ2e[trait]) = (0.056842881348442224,0.9423649400242309)
  1.122919 seconds (23.28 M allocations: 379.506 MB, 22.47% gc time)


### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.


In [66]:
# heritability and standard errors
[hST'; hST_se']

2x13 Array{Float64,2}:
 0.261041  0.188742  0.318572  0.265569  …  0.164868  0.0829866  0.0568429
 0.261041  0.188742  0.318572  0.265569     0.164868  0.0829866  0.0568429

## Pairwise traits

Following code snippet calculates joint heritability among all pairs of traits, a total of 78 bivariate variane component models.

In [2]:
# additive genetic effects (2x2 psd matrices) from bavariate trait analysis;
Σa = Array{Matrix{Float64}}(13, 13)
# environmental effects (2x2 psd matrices) from bavariate trait analysis;
Σe = Array{Matrix{Float64}}(13, 13)

@time for i in 1:13
    for j in (i+1):13
        println(names(cg10k_trait)[i + 2], names(cg10k_trait)[j + 2])
        # form data set for (trait1, trait2)
        traitij_data = TwoVarCompVariateRotate(cg10kdata_rotated.Yrot[:, [i;j]], cg10kdata_rotated.Xrot, 
            cg10kdata_rotated.eigval, cg10kdata_rotated.logdetV2)
        # initialize model parameters
        traitij_model = VarianceComponentModel(traitij_data)
        # estimate variance components
        mle_fs!(traitij_model, traitij_data; solver=:Knitro, verbose=false)
        Σa[i, j] = traitij_model.Σ[1]
        Σe[i, j] = traitij_model.Σ[2]
        @show Σa[i, j], Σe[i, j]
    end
end

Trait1Trait2
(Σa[i,j],Σe[i,j]) = (
[0.2601194348964541 0.17621582512277895
 0.17621582512277895 0.18737615485752512],

[0.7365894054973976 0.5838920950970753
 0.5838920950970753 0.8120331390127362])
Trait1Trait3
(Σa[i,j],Σe[i,j]) = 

### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.


(
[0.2615639935926094 -0.013126817780793214
 -0.013126817780793214 0.31905662257097117],

[0.735180211126028 -0.12112674792808123
 -0.12112674792808123 0.6796789899029853])
Trait1Trait4
(Σa[i,j],Σe[i,j]) = 

### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.


(
[0.26087960318141157 0.22261440535048016
 0.22261440535048016 0.2655808327657969],

[0.735845998143173 0.5994353346472087
 0.5994353346472087 0.7303474474454928])
Trait1Trait5
(Σa[i,j],Σe[i,j]) = 

...

### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.


(
[0.16488293344606353 -0.0015841370683006104
 -0.0015841370683006104 0.05749684397777908],

[0.833797948821629 0.20061222756374675
 0.20061222756374675 0.9417152318762243])
Trait12Trait13
(Σa[i,j],Σe[i,j]) = 

### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.


(
[0.08459517874933027 0.06850620327544214
 0.06850620327544214 0.0554771581601963],

[0.9142134905816461 0.5731514383815555
 0.5731514383815555 0.943733788071119])
 86.423563 seconds (1.92 G allocations: 30.350 GB, 4.91% gc time)


### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.


## 5-trait analysis

Researchers want to jointly analyze traits 5-9. Our strategy is to try both Fisher scoring and MM algorithm with different starting point, and choose the best local optimum.

In [45]:
traitidx = 5:9
# form data set
trait59_data = TwoVarCompVariateRotate(cg10kdata_rotated.Yrot[:, traitidx], cg10kdata_rotated.Xrot, 
    cg10kdata_rotated.eigval, cg10kdata_rotated.logdetV2)
# initialize model parameters
trait59_model = VarianceComponentModel(trait59_data)
# estimate variance components
@time mle_fs!(trait59_model, trait59_data; solver=:Knitro, verbose=true)
trait59_model


Knitro 10.1.0 STUDENT LICENSE (problem size limit = 300)

            Student License
       (NOT FOR COMMERCIAL USE)
         Artelys Knitro 10.1.0

Knitro presolve eliminated 0 variables and 0 constraints.

algorithm:            1
The problem is identified as bound constrained only.
Knitro changing bar_initpt from AUTO to 3.
Knitro changing bar_murule from AUTO to 4.
Knitro changing bar_penaltycons from AUTO to 1.
Knitro changing bar_penaltyrule from AUTO to 2.
Knitro changing bar_switchrule from AUTO to 1.
Knitro changing linsolver from AUTO to 2.

Problem Characteristics                    ( Presolved)
-----------------------
Objective goal:  Maximize
Number of variables:                    30 (        30)
    bounded below:                      10 (        10)
    bounded above:                       0 (         0)
    bounded below and above:             0 (         0)
    fixed:                               0 (         0)
    free:                               20 (        20)

### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.


VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}(0x5 Array{Float64,2},(
5x5 Array{Float64,2}:
 0.293285  0.286273  0.246039   0.156834   0.265501
 0.286273  0.287347  0.224065   0.187732   0.254287
 0.246039  0.224065  0.252785   0.0431791  0.191771
 0.156834  0.187732  0.0431791  0.259103   0.174374
 0.265501  0.254287  0.191771   0.174374   0.377391,

5x5 Array{Float64,2}:
 0.705656  0.65533    0.661038    0.22789    0.497238
 0.65533   0.712524   0.57635     0.435353   0.462432
 0.661038  0.57635    0.74447    -0.0140503  0.493485
 0.22789   0.435353  -0.0140503   0.747925   0.125282
 0.497238  0.462432   0.493485    0.125282   0.74239 ))

We see Fisher scoring is having trouble finding the optimum. Let's try the MM algorithm, starting from the Fisher scoring answer.

In [46]:
# trait59_model contains the fitted model by Fisher scoring now
@time mle_mm!(trait59_model, trait59_data; verbose=true)
trait59_model


     MM Algorithm
  Iter      Objective  
--------  -------------
       0   8.705871e+03
       1   8.757960e+03
       2   8.773751e+03
       3   8.778653e+03
       4   8.780308e+03
       5   8.780982e+03
       6   8.781350e+03
       7   8.781619e+03
       8   8.781851e+03
       9   8.782068e+03
      10   8.782277e+03
      20   8.784119e+03
      30   8.785626e+03
      40   8.786891e+03
      50   8.787989e+03
      60   8.788997e+03
      70   8.789999e+03
      80   8.791101e+03
      90   8.792448e+03
     100   8.794239e+03
     110   8.796745e+03
     120   8.800327e+03
     130   8.805422e+03
     140   8.812485e+03
     150   8.821859e+03
     160   8.833572e+03
     170   8.847171e+03
     180   8.861722e+03
     190   8.876067e+03
     200   8.889196e+03
     210   8.900503e+03
     220   8.909810e+03
     230   8.917248e+03
     240   8.923090e+03
     250   8.927647e+03
     260   8.931201e+03
     270   8.933986e+03
     280   8.936185e+03
     290   8.937938e+

VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}(0x5 Array{Float64,2},(
5x5 Array{Float64,2}:
 0.232505   0.207127   0.213454   0.0663597  0.218199 
 0.207127   0.185167   0.188206   0.0627829  0.190923 
 0.213454   0.188206   0.205745   0.0447439  0.20381  
 0.0663597  0.0627829  0.0447439  0.0464538  0.0517928
 0.218199   0.190923   0.20381    0.0517928  0.235715 ,

5x5 Array{Float64,2}:
 0.764114  0.732092   0.692173    0.316228   0.534003
 0.732092  0.811758   0.612219    0.555289   0.519043
 0.692173  0.612219   0.79091    -0.0148224  0.475712
 0.316228  0.555289  -0.0148224   0.951042   0.244044
 0.534003  0.519043   0.475712    0.244044   0.762242))

Do another run of MM algorithm from default starting point. It leads to a better local optimum. Follow up anlaysis should use this result.

In [47]:
# default starting point
trait59_model = VarianceComponentModel(trait59_data)
@time _, _, _, Σcov, = mle_mm!(trait59_model, trait59_data; verbose=true)
trait59_model


     MM Algorithm
  Iter      Objective  
--------  -------------
       0  -5.042149e+04
       1  -1.699687e+04
       2  -1.543968e+03
       3   5.143609e+03
       4   7.669359e+03
       5   8.489390e+03
       6   8.728893e+03
       7   8.796019e+03
       8   8.815880e+03
       9   8.823279e+03
      10   8.827419e+03
      20   8.854389e+03
      30   8.874943e+03
      40   8.890795e+03
      50   8.903062e+03
      60   8.912619e+03
      70   8.920130e+03
      80   8.926088e+03
      90   8.930861e+03
     100   8.934721e+03
     110   8.937872e+03
     120   8.940467e+03
     130   8.942622e+03
     140   8.944425e+03
     150   8.945946e+03
     160   8.947237e+03
     170   8.948339e+03
     180   8.949287e+03
     190   8.950106e+03
     200   8.950818e+03
     210   8.951439e+03
     220   8.951983e+03
     230   8.952462e+03
     240   8.952886e+03
     250   8.953262e+03
     260   8.953597e+03
     270   8.953896e+03
     280   8.954164e+03
     290   8.954404e+

VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}(0x5 Array{Float64,2},(
5x5 Array{Float64,2}:
 0.28408   0.282346  0.235651   0.163231   0.243528
 0.282346  0.287797  0.222888   0.189209   0.231229
 0.235651  0.222888  0.215833   0.0895747  0.212778
 0.163231  0.189209  0.0895747  0.199473   0.107541
 0.243528  0.231229  0.212778   0.107541   0.246379,

5x5 Array{Float64,2}:
 0.714039  0.65898    0.67085     0.221737   0.509435
 0.65898   0.711746   0.578905    0.431453   0.480001
 0.67085   0.578905   0.781276   -0.0579601  0.467102
 0.221737  0.431453  -0.0579601   0.800138   0.189943
 0.509435  0.480001   0.467102    0.189943   0.751844))

Heritability from 5-variate estimate and their standard errors.

In [49]:
h, hse = heritability(trait59_model.Σ, Σcov)
[h'; hse']

2x5 Array{Float64,2}:
 0.284615   0.287928   0.216459  0.199551   0.246817 
 0.0774592  0.0770454  0.08369   0.0850318  0.0809451

## 13-trait analysis

Researchers would like to jointly analyze all 13 traits. Fisher scoring algorithm (with KNITRO backend) again has trouble finding the optimum, ending with `OptError` of 2.11e+03. 

In [51]:
# initialize model parameters
alltrait_model = VarianceComponentModel(cg10kdata_rotated)
# estimate variance components
@time mle_fs!(alltrait_model, cg10kdata_rotated; solver=:Knitro, verbose=true)
alltrait_model


Knitro 10.1.0 STUDENT LICENSE (problem size limit = 300)

            Student License
       (NOT FOR COMMERCIAL USE)
         Artelys Knitro 10.1.0

Knitro presolve eliminated 0 variables and 0 constraints.

algorithm:            1
The problem is identified as bound constrained only.
Knitro changing bar_initpt from AUTO to 3.
Knitro changing bar_murule from AUTO to 4.
Knitro changing bar_penaltycons from AUTO to 1.
Knitro changing bar_penaltyrule from AUTO to 2.
Knitro changing bar_switchrule from AUTO to 1.
Knitro changing linsolver from AUTO to 2.

Problem Characteristics                    ( Presolved)
-----------------------
Objective goal:  Maximize
Number of variables:                   182 (       182)
    bounded below:                      26 (        26)
    bounded above:                       0 (         0)
    bounded below and above:             0 (         0)
    fixed:                               0 (         0)
    free:                              156 (       156)

### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.


VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}(0x13 Array{Float64,2},(
13x13 Array{Float64,2}:
  0.314349    0.132106     -0.0354149  …  -0.136163    -0.1057    
  0.132106    0.139838      0.14053       -0.01739     -0.0045432 
 -0.0354149   0.14053       0.356969       0.0760162    0.092202  
  0.3435      0.0544454    -0.205904      -0.179036    -0.150632  
 -0.129431    0.0162413     0.18923        0.0740475    0.0695918 
 -0.113559    0.00982848    0.160951   …   0.0670134    0.0576479 
 -0.103943    0.0127981     0.167589       0.0754349    0.0715262 
 -0.0549323   0.000389919   0.0502953      0.00850429   0.00325788
 -0.205308   -0.0387772     0.154017       0.085873     0.0790097 
 -0.147019   -0.085665     -0.064819       0.127711     0.0730854 
 -0.155357   -0.0717892    -0.0087627  …   0.0946257    0.0511083 
 -0.136163   -0.01739       0.0760162      0.357154     0.236549  
 -0.1057     -0.0045432     0.092202       0.236549     

Re-run using MM algorithm, which locates a better mode. Follow up anlaysis should use this result.

In [53]:
# initialize model parameters
alltrait_model = VarianceComponentModel(cg10kdata_rotated)
# estimate variance components
@time _, _, _, Σcov, = mle_mm!(alltrait_model, cg10kdata_rotated; verbose=true)
alltrait_model


     MM Algorithm
  Iter      Objective  
--------  -------------
       0  -1.311337e+05
       1  -8.002195e+04
       2  -5.807051e+04
       3  -4.926234e+04
       4  -4.611182e+04
       5  -4.511727e+04
       6  -4.482798e+04
       7  -4.474410e+04
       8  -4.471610e+04
       9  -4.470285e+04
      10  -4.469355e+04
      20  -4.462331e+04
      30  -4.456960e+04
      40  -4.452834e+04
      50  -4.449652e+04
      60  -4.447178e+04
      70  -4.445237e+04
      80  -4.443699e+04
      90  -4.442467e+04
     100  -4.441470e+04
     110  -4.440656e+04
     120  -4.439985e+04
     130  -4.439427e+04
     140  -4.438959e+04
     150  -4.438564e+04
     160  -4.438229e+04
     170  -4.437941e+04
     180  -4.437694e+04
     190  -4.437480e+04
     200  -4.437294e+04
     210  -4.437131e+04
     220  -4.436989e+04
     230  -4.436863e+04
     240  -4.436751e+04
     250  -4.436652e+04
     260  -4.436564e+04
     270  -4.436485e+04
     280  -4.436414e+04
     290  -4.436351e+

VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}(0x13 Array{Float64,2},(
13x13 Array{Float64,2}:
  0.273498    0.192141    -0.0207392   0.231615   …  -0.128643   -0.098307  
  0.192141    0.219573     0.134389    0.0800317     -0.0687355  -0.0433724 
 -0.0207392   0.134389     0.329149   -0.158086       0.0717258   0.097381  
  0.231615    0.0800317   -0.158086    0.277921      -0.129316   -0.104571  
 -0.149294   -0.0130248    0.186275   -0.219746       0.0673901   0.0798606 
 -0.131878   -0.00396265   0.169037   -0.204038   …   0.0539454   0.0662375 
 -0.14514    -0.0332988    0.168384   -0.190066       0.0778299   0.0867255 
 -0.0299641   0.0372326    0.0617453  -0.0918129     -0.0120473  -0.00309877
 -0.205216   -0.0839698    0.137149   -0.232933       0.0895106   0.0971193 
 -0.100475   -0.114933    -0.088079   -0.0435633      0.046928   -0.0048387 
 -0.133733   -0.109897    -0.0247458  -0.0985453  …   0.0587572   0.0114353 
 -0.128643   

Heritability estimate from 13-variate analysis.

In [54]:
h, hse = heritability(alltrait_model.Σ, Σcov)
[h'; hse']

2x13 Array{Float64,2}:
 0.274338  0.219669   0.329511   0.279019   …  0.172946  0.117315   0.10642  
 0.07785   0.0818365  0.0727477  0.0775928     0.087259  0.0902116  0.0903971

## Save analysis results

In [16]:
#@save "copd.jld"
whos()

                 #130#wsession    280 bytes  JLD.JldWriteSession
                 #132#wsession    280 bytes  JLD.JldWriteSession
                 #159#wsession    280 bytes  JLD.JldWriteSession
                    ArrayViews    184 KB     Module
                          Base  34997 KB     Module
                       BinDeps    208 KB     Module
                         Blink    102 KB     Module
                         Blosc     37 KB     Module
                        Codecs     25 KB     Module
                    ColorTypes    323 KB     Module
                        Colors    764 KB     Module
                        Compat    315 KB     Module
                         Conda     65 KB     Module
                          Core   6865 KB     Module
                    DataArrays    784 KB     Module
                    DataFrames   1837 KB     Module
                        Docile    414 KB     Module
                        FileIO    538 KB     Module
             FixedPointNu